In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from finitediff.grid import adapted_grid, plot_convergence
from finitediff.tests.test_grid import g, g2
%matplotlib inline

In [ ]:
plot_convergence('grid_additions', [(32,), (16, 16), (8, 8, 8, 8)], g)

In [ ]:
plot_convergence('grid_additions', [(64,), (32, 32), (48, 12, 4)], g)

In [ ]:
blr = ((.3, .1), (.3, .1))
plot_convergence('grid_additions', [(64,), (32, 32), (48, 12, 4)], g, blurs=blr)

In [ ]:
plot_convergence('grid_additions', [(64,), (32, 32), (32, 24, 4, 4)], g, blurs=blr)

In [ ]:
plot_convergence('grid_additions', [(32,)*2, (16,)*4, (8,)*8], g, blurs=blr)

In [ ]:
plot_convergence('grid_additions', [(32,), (16,)*2, (8,)*4, (4,)*8], g, blurs=blr)

In [ ]:
plot_convergence('grid_additions', [(48,), (24,)*2, (12,)*4, (16,)*3], g)

In [ ]:
plot_convergence('grid_additions', [(32,), (16,)*2, (8,)*4, (4,)*8], g2)

In [ ]:
plot_convergence('grid_additions', [(32,), (16,)*2, (8,)*4, (4,)*8], g2, ntrail=3)

In [ ]:
plot_convergence('grid_additions', [(32, 32), (16,)*4], g2)

In [ ]:
plot_convergence('grid_additions', [(32, 32), (16,)*4], g2, ntrail=3, blurs=blr)

In [ ]:
plot_convergence('grid_additions', [(64,)*2, (16,)*8], g2, ntrail=3, blurs=blr)

In [ ]:
def gs(x):
    return [(a, b) for a, b in zip(g(x), g2(x))]
plot_convergence('grid_additions', [(64,)*2, (16,)*8], gs, ntrail=3, blurs=blr,
                 metric=lambda x: x[0]+x[1])

In [ ]:
plot_convergence('ntrail', [2, 3, 4], g)

In [ ]:
plot_convergence('blurs', [((.5,), (.5,)), ((.5, .25), (.5, .25)), ((.125,), (.125,))], g)

In [ ]:
plot_convergence('blurs', [((.5,), (.5,)), ((.5, .25), (.5, .25)), ((.125,), (.125,))],
                 g, ntrail=4)